### Connecting to a Database

#### Meet SQLAlchemy

- Two main pieces
	- Core (relational model focused)
	- ORM (user data model focused)

#### Connecting

~~~
from sqlalchemy import create_engine

engine = create_engine('sqlite://census_nyc.sqlite')

connection = engine.connect()
~~~

- Engine: common interface to the database from SQLAlchemy
- Connection string: all the details required to find the  database (and login, if necessary)
	- 'sqlite://census_nyc.sqlite': driver+dialect + '///' + filename

- Before querying your database, you'll want to know what is in it: what the tables are, for example:

~~~
print(engine.table_names())
~~~

#### Reflection

- reads database and builds SQLAlchemy Table objects

~~~
from sqlalchemy import MetaData, Table

metadata = MetaData()

census = Table('census',metadata,autoload=True,autoload_with=engine)

print(repr(census))
~~~

### Basic SQL querying

- SELECT column_name FROM table_name

~~~
stmt = 'SELECT * FROM people'

result_proxy = connection.execute(stmt) # ResultProxy

results = result_proxy.fetchall() # ResultSet
~~~

#### Handling ResultSets

~~~
first_row = results[0]
~~~

#### SQLAlchemy querying

~~~
from sqlalchemy import Table, MetaData

...

metadata = MetaData()

census = Table('census',metadata,autoload=True,autoload_with=engine)

stmt = select([census])

results = connection.execute(stmt).fetchall()
~~~

**SQLAlchemy Select Statement**

- Requires a list of one or more Tables or Columns
- Using a table will select all the columns in it


#### Where clauses

~~~
stmt = select([census])

stmt = stmt.where(census.columns.state == 'California')

results = connection.execute(stmt).fecthall()

for result in results:
	print(result.state, result.age)
~~~

- Restrict data returned by a query based on boolean conditions
- Compare a column against a value or another column
- Often used comparisons: '==', '<=', '>=' or '!='

**Expressions**

- Provide more complex conditions than simple operators
- Eg. in_(), like(), between()
- Available as methods on a Column

~~~
stmt = select([census])

stmt = stmt.where(census.columns.state.startswith('New'))

for result in connection.execute(stmt):
	print(results.state,result.pop2000)
~~~

**Conjunctions**

- Allow us to have multiple criteria in a where clause
- Eg. and_(), not_(), or_()

~~~
from sqlalchemy import or_

stmt = select([census])

stmt = stmt.where(or_(census.columns.state=='California',census.columns.state=='New York'))

for result in connection.execute(stmt):
	print(result.state,results.sex)
~~~

~~~
# Create a query for the census table: stmt
stmt = select([census])

# Append a where clause to match all the states in_ the list states
stmt = stmt.where(census.columns.state.in_(states))

# Loop over the ResultProxy and print the state and its population in 2000
for res in connection.execute(stmt):
    print(res.state, res.pop2000)
~~~

~~~
# Import and_
from sqlalchemy import and_

# Build a query for the census table: stmt
stmt = select([census])

# Append a where clause to select only non-male records from California using and_
stmt = stmt.where(
    # The state of California with a non-male sex
    and_(census.columns.state == 'California',
         census.columns.sex != 'M'
         )
)

# Loop over the ResultProxy printing the age and sex
for result in connection.execute(stmt):
    print(result.age, result.sex)
~~~

#### Order by clauses

- Allows us to contorl the order in which records are returned in the query results
- Available as a method on statements *order_by()*

~~~
stmt = select([census.columns.state])

stmt = stmt.order_by(census.columns.state)

results = connection.execute(stmt).fetchall()
~~~

- Order by Descending: wrap the column with *desc()* in the *order_by()* clause

- Order by Multiple: just separate multiple columns with a comma
	- Orders completely by the first column
	- Then, if there are duplicates in the first column, orders by the second column
	- repeat until all columns are ordered

~~~
stmt = select([census.columns.state,
		census.columns.sex])

stmt = stmt.order_by(census.columns.state,
		census.columns.sex)
~~~

#### SQL functions

- Eg. Count, Sum
- from sqlalchemy import func
- More efficient than processing in Python
- Aggregate data

**Sum example**

~~~
from sqlalchemy import func

stmt = select([func.sum(census.columns.pop2008)])

res = connection.execute(stmt).scalar()
~~~

**Group by**

- Allows us to group rows by common values

~~~
stmt = select([census.columns.sex,func.sum(census.columns.pop2008)])

stmt = stmt.group_by(census.columns.sex)

results = connection.execute(stmt).fetchall()
~~~

- Supports multiple columns to group by with a pattern similar to *order_by()*
- Requires all selected columns to be grouped or aggregated by a function

- Group by Multiple:

~~~
stmt = select([census.columns.sex,census.columns.age,func.sum(census.columns.pop2008)])

stmt = stmt.group_by(census.columns.sex,census.columns.age)

results = connection.execute(stmt).fetchall()
~~~

**Handling ResultSets from Functions**

- SQLAlchemy autogenerates 'columns name' for functions in the ResultSet
- The column names are often func_#, such as count_1
- Replace them with the label() method

~~~
stmt = select([census.columns.sex,func.sum(census.columns.pop2008).label('pop2008_sum')])

stmt = stmt.group_by(census.columns.sex)

res = connection.execute(stmt).fetchall()

print(res[0].keys())
~~~

~~~
# Build a query to count the distinct states values: stmt
stmt = select([func.count(census.columns.state.distinct())])

# Execute the query and store the scalar result: distinct_state_count
distinct_state_count = connection.execute(stmt).scalar()

# Print the distinct_state_count
print(distinct_state_count)
~~~

~~~
# Import func
from sqlalchemy import func

# Build an expression to calculate the sum of pop2008 labeled as population
pop2008_sum = func.sum(census.columns.pop2008).label('population')

# Build a query to select the state and sum of pop2008: stmt
stmt = select([census.columns.state, pop2008_sum])

# Group stmt by state
stmt = stmt.group_by(census.columns.state)

# Execute the statement and store all the records: results
results = connection.execute(stmt).fetchall()

# Print results
print(results)

# Print the keys/column names of the results returned
print(results[0].keys())
~~~

#### SQLAlchemy + Matplotlib

~~~
df = pd.DataFrame(results)

df.columns = results[0].keys()
~~~

#### Math operators

- addition +
- subtraction -
- multiplication *
- division /
- modulus %

**Calculating difference**

~~~
stmt = select([census.columns.age,
		(census.columns.pop2008-census.columns.pop2000).label('pop_change')])

stmt = stmt.group_by(census.columns.age)

stmt = stmt.order_by(desc('pop_change'))

stmt = stmt.limit(5) # first 5 results

results = connection.execute(stmt).fetchall()
~~~

#### Case statement

- Used to treat data differently based on a condition
- Accepts a list of conditions to match and a column to return if the condition matches
- The list of conditions ends with an else clause to determine what to do when a record doesn't match any prior conditions

~~~
from sqlalchemy import case

stmt = select([
		func.sum(
			case([
				(census.columns.state=='New York',
				census.columns.pop2008)
			],else_=0))])

results = connection.execute(stmt).fetchall()
~~~

#### Cast Statement

- Converts data to another type
- Useful for converting
	- integers to floats for division
	- string to dates and times
- Accepts a column or expression and the target Type

**Percentage example**

~~~
from sqlalchemy import case, cast, Float

stmt = select([
		(func.sum(
			case([
				census.columns.state=='New York',
				census.columns.pop2008)
			], else_=0)) /
		cast(func.sum(census.columns.pop2008),
			Float) * 100).label('ny_percent')])

res = con.execute(stmt).fetchall()
~~~

~~~
# Import create_engine function
from sqlalchemy import create_engine

# Create an engine to the census database
engine = create_engine('mysql+pymysql://' + 'student:datacamp' + '@courses.csrrinzqubik.us-east-1.rds.amazonaws.com:3306/' + 'census')
# Dialect+Driver + Username+Password + Host+Port + DBName

# Print the table names
print(engine.table_names())
~~~

~~~

# import case, cast and Float from sqlalchemy
from sqlalchemy import case, cast, Float

# Build an expression to calculate female population in 2000
female_pop2000 = func.sum(
    case([
        (census.columns.sex == 'F', census.columns.pop2000)
    ], else_=0))

# Cast an expression to calculate total population in 2000 to Float
total_pop2000 = cast(func.sum(census.columns.pop2000), Float)

# Build a query to calculate the percentage of females in 2000: stmt
stmt = select([female_pop2000 / total_pop2000* 100])

# Execute the query and store the scalar result: percent_female
percent_female = connection.execute(stmt).scalar()

# Print the percentage
print(percent_female)
~~~

#### Relationships

- Allow us to avoid duplicate data
- Make it easy to change things in one place
- Useful to break out information from a table we don't need very often

**Automatic Joins**

~~~
stmt = select([census.columns.pop2008,state_fact.columns.abbreviation])

res = con.execute(stmt).fetchall()
~~~

#### Join

- Accepts a Table and an optional expression that explains how the two tables are related
- The expression is not needed if the relationship is predefined and available via reflection
- Comes immediately after the *select()* clause and prior to any *where()*, *order_by()* or *group_by()* clauses

**Select_from**

- Used to replace the defualt, derived FROM clause with a join
- wraps the *join()* clause

~~~
stmt = select([func.sum(census.columns.pop2000)])

stmt = stmt.select_from(census.join(state_fact))

stmt = stmt.where(state_fact.columns.circuit_court=='10')

res = con.execute(stmt).scalar()
~~~

**Joining Tables without Predefined Relaaionship**

- Join accepts a Table and a an optional expression that explains how the two tables are related
- Will only join on data that match between the two columns
- Avoid joining on columns of different types

~~~
stmt = select([func.sum(census.columns.pop2008)])

stmt = stmt.select_from(
	census.join(state_fact, census.columns.state == state_fact.columns.name)

stmt = stmt.where(
	state_fact.columns.census_division_name == 'East South Central')

res = con.execute(stmt).fetchall()
~~~

~~~
# Build a statement to select the state, sum of 2008 population and census
# division name: stmt
stmt = select([
    census.columns.state,
    func.sum(census.columns.pop2008),
    state_fact.columns.census_division_name
])

# Append select_from to join the census and state_fact tables by the census state and state_fact name columns
stmt = stmt.select_from(
    census.join(state_fact, census.columns.state == state_fact.columns.name)
)

# Append a group by for the state_fact name column
stmt = stmt.group_by(state_fact.columns.name)

# Execute the statement and get the results: results
results = connection.execute(stmt).fetchall()

# Loop over the the results object and print each record.
for record in results:
    print(record)
~~~

#### Hierarchical Tables

- Contain a relationship with themselves
- Commonly found in: organizational, geographic, newtork, graph

**alias()**

- Requires a way to view the table via multiple names
- Creates a unique reference that we can use

~~~
managers = employees.alias()

stmt = select(
	[managers.columns.name.label('manager'),
	employees.columns.name.label('employee')])

stmt = stmt.select_from(employees.join(
	managers, managers.columns.id == employees.columns.manager))

stmt = stmt.order_by(managers.columns.name)
~~~

**Group_by and Func**

- It's important to target *group_by()* at the right alias
- Be careful with what you perform functions on
- If you don't find yourself using both the alias and the table name for a query, don't create the alias at all

~~~
managers = employees.alias()

stmt = select([managers.columns.name,func.sum(employees.columns.sal)])

stmt = stmt.select_from(employees.join(managers, managers.columns.id == employees.columns.manager))

stmt = stmt.group_by(managers.columns.name)
~~~

#### Dealing with Large ResultSets

- *fetchmany()* lets us specify how many rows we want to act upon
- We can loop over *fetchmany()*
- It returns an empty list when there are no more records
- We have to close the ResultProxy afterwards

~~~
while more_results:
	partial_results = result_proxy.fetchmany(50)

	more_results = not(partial_results == [])

	for row in partial_results:
		state_count[row.state] += 1

result_proxy.close()
~~~

#### Creating Databases

- Varies by the database type
- Databases like PostgreSQL and MySQL have command line tools to initialize the DB
- With SQLite, the *create_engine()* statement will create the DB and file if they do not already exist

~~~
from sqlalchemy import (Table, Column, String, Integer, Decimal, Boolean)

employees = Table('employees',metadata,
		Column('id', Integer()),
		Column('name', String(255)),
		Column('salary', Decimal()),
		Column('active', Boolean()))

metadata.create_all(engine)

print(engine.table_names())
~~~

**Creating tables**

- Still uses the Table object like we did for reflection
- Replaces the autoload keyword arguments with Column objects
- Creates the tables in the actual database by using the *create_all()* method on the MetaData instance
- You need to use other tools to handle database table updates, such as Alembic or raw SQL

**Additional Column Options**

- *unique* forces all values for the data in a column to be unique
- *nullable* determines if a column can be empty in a row
- *default* sets a default value if one isn't supplied

~~~
employees = Table('employees',metadata,
		Column('id', Integer()),
		Column('name', String(255), unique=True, nullable=False),
		Column('salary', Decimal(), default=100.00),
		Column('active', Boolean()), default=True)

print(employees.constraints)
~~~

#### Adding Data to a Table

- Done with *insert()* statement
- *Insert()* takes the table we are loading into as the argument
- We add all the values we want to insert in with the values clause as column=value pairs
- Doesn't return any rows, so no need for a fetch method

**Inserting one row**

~~~
from sqlalchemy import insert

stmt = insert(employees).values(id=1,name='Jason',salary=1.00,active=True)

result_proxy = con.execute(stmt)

print(result_proxy.rowcount) # prints 1
~~~

**Inserting multiple rows**

- build an insert statement without any values
- build a list of dictionaries that represent all the values clauses for the rows you want to insert
- pass both the *stmt* and the values list to the execute method on connection

~~~
stmt = insert(employees)

values_list = [
	{'id': 2, 'name': 'Rebecca', 'salary': 2.00, 'active': True},
	{'id': 3, 'name': 'Bob', 'salary': 0.00, 'active': False}
	]

result_proxy = con.execute(stmt, values_list)

print(result_proxy.rowcount) # prints 2
~~~

#### Updating Data in a Table

- Done with the *update* statement
- Similar to the insert statement but includes a *where* clause to determine what record will be updated
- We add all the values we want to update with the *values* clause as column=value pairs

**Updating one row**

~~~
from sqlalchemy import update

stmt = update(employees)

stmt = stmt.where(employees.columns.id==3)

stmt = stmt.values(active=True)

result_proxy = con.execute(stmt)

print(result_proxy.rowcount) # prints 1
~~~

**Updating multiple rows**

- build a where clause that will select all the records you want to update

~~~
stmt = update(employees)

stmt = stmt.where(employees.columns.active==True)

stmt = stmt.values(active=False,salary=0.00)

result_proxy = con.execute(stmt)
~~~

**Correlated updates**

~~~
new_salary = select([employees.columns.salary])

new_salary = new_salary.order_by(desc(employees.columns.salary))

new_salary = new_salary.limit(1)

stmt = update(employees)

stmt = stmt.values(salary=new_salary)

result_proxy = con.execute(stmt)
~~~

- uses a *select()* statement to find the value for the column we are updating
- commonly used to update records to a maximum value or change a string to match an abbreviation from another table

~~~

# Build a statement to select name from state_fact: stmt
fips_stmt = select([state_fact.columns.name])

# Append a where clause to Match the fips_state to flat_census fips_code
fips_stmt = fips_stmt.where(
    state_fact.columns.fips_state == flat_census.columns.fips_code)

# Build an update statement to set the name to fips_stmt: update_stmt
update_stmt = update(flat_census).values(state_name=fips_stmt)

# Execute update_stmt: results
results = connection.execute(update_stmt)

# Print rowcount
print(results.rowcount)

~~~

#### Deleting all Data from a Table

~~~
from sqlalchemy import delete

stmt = select([func.count(extra_employees.columns.id)])

print(con.execute(stmt).scalar())

delete_stmt = delete(extra_employees)

result_proxy = con.execute(delete_stmt)

print(result_proxy.rowcount)
~~~

#### Deleting Specific Rows

- build a where clause that will select all the records you want to delete

~~~
stmt = delete(employees).where(employees.columns.id == 3)

result_proxy = con.execute(stmt)

print(resul_proxy.rowcount)
~~~

#### Dropping a Table completely

- uses the *drop* method
- accepts the engine as an argument so it knows where to remove the table from
- won't remove it from the metadata until the python process  is restarted

~~~
extra_employees.drop(engine)

print(extra_employees.exists(engine)) # print False
~~~

#### Dropping all the Tables

- uses the *drop_all()* method on MetaData

~~~
metadata.drop_all(engine)

engine.table_names() # returns []
~~~

~~~

# Build a statement to count records using the sex column for Men ('M') age 36: stmt
stmt = select([func.count(census.columns.sex)]).where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the select statement and use the scalar() fetch method to save the record count
to_delete = connection.execute(stmt).scalar()

# Build a statement to delete records from the census table: stmt_del
stmt_del = delete(census)

# Append a where clause to target Men ('M') age 36
stmt_del = stmt_del.where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the statement: results
results = connection.execute(stmt_del)

# Print affected rowcount and to_delete record count, make sure they match
print(results.rowcount, to_delete)

~~~

~~~
# Import select
from sqlalchemy import select

# Calculate weighted average age: stmt
stmt = select([census.columns.sex,
               (func.sum(census.columns.pop2008 * census.columns.age) /
                func.sum(census.columns.pop2008)).label('average_age')
               ])

# Group by sex
stmt = stmt.group_by(census.columns.sex)

# Execute the query and store the results: results
results = connection.execute(stmt).fetchall()

# Print the average age by sex
for sex, avg_age in results:
    print(sex, avg_age)
    
~~~

~~~

# import case, cast and Float from sqlalchemy
from sqlalchemy import case, cast, Float

# Build a query to calculate the percentage of females in 2000: stmt
stmt = select([census.columns.state,
    (func.sum(
        case([
            (census.columns.sex == 'F',census.columns.pop2000)
        ], else_=0)) /
     cast(func.sum(census.columns.pop2000), Float) * 100).label('percent_female')
])

# Group By state
stmt = stmt.group_by(census.columns.state)

# Execute the query and store the results: results
results = connection.execute(stmt).fetchall()

# Print the percentage
for result in results:
    print(result.state, result.percent_female)

~~~